In [6]:
import numpy as np
import sympy as sp
from sympy import Symbol
from sympy import solve, Poly, Eq, Function, exp
import math
import matplotlib.pyplot as plt
from numpy.lib.scimath import sqrt as csqrt
from scipy.signal import find_peaks
from scipy.constants import lambda2nu, nu2lambda, c
import os
import imp
import matplotlib.pyplot as plt
import sys

# Moises Path

#windows path
#os.add_dll_directory('C:\\Program Files\\Lumerical\\v202\\api\\python\\') 
#lumapi = imp.load_source('lumapi','C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')

#linux path
#sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
#import lumapi
#sys.path.append(os.path.dirname('/home/moises/Downloads/'))

# Leonardo Path

###############################################################
#LINUX PATH

#sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
#import lumapi
#sys.path.append(os.path.dirname('/media/leonardo/datam02/')) #Current directory

###############################################################
#WINDOWS PATH

spec_win = imp.util.spec_from_file_location('lumapi', 'C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')
lumapi = imp.util.module_from_spec(spec_win)
os.add_dll_directory("C:/Program Files/Lumerical/v202/api/python")
sys.path.append(os.path.dirname('C:/Users/Léo/Documents/Facul/Capacitação/GitHub/2x4MMI_Hybrid'))
spec_win.loader.exec_module(lumapi)

In [12]:
fdtd = lumapi.FDTD(filename = '../CML/Hybrid.fsp')

## parameters

In [61]:
core = 'GaInAsP'
cladding = 'InP'
buffer = 'InP'
substrate = 'InP'
n = 3.2515 ## refractive index of GaInAsP for 1.55 um
w = 2e-6
thickness = 0.3e-6
Wmf = 18.6e-6
Wms = 9.02e-6
Ldevice = 227e-6
LS24 = 116e-6
LM22 = 111e-6
gap = 1.1e-6
c = 299792458
xspan = 10e-6
yspan = 5e-6
v = c/n

In [126]:
fdtd.selectall()
fdtd.delete()

# Creating MMIs
fdtd.addobject('linear_taper') # Add MMI 2x4 Box
fdtd.set({
    'name':'2X4',
    'x':0,
    'y':0,
    'z':0,
    'thickness':thickness,
    'angle_side':90,
    'width_l':Wms,
    'width_r':Wmf,
    'len':LS24,
    'material':core
})

fdtd.addrect() # Add MMI 2x2 Box
fdtd.set({
    'name'     : 'MMI2x2',
    'material' : core,
    'y span'   : Wms,
    'z span'   : thickness,
    'z'        : 0,
    'x min'    : LS24/2,
    'x max'    : LM22 + LS24/2,
    'y min'    : -Wmf/2,
    'y max'    : 0
})

# Creating input bends 
fdtd.unselectall()
fdtd.addobject('s_bend_wg') # Creating the bend
fdtd.addtogroup('Input Bends') # Adding the bends on the same group
fdtd.set({'name' :'bend up',
    'first axis' :'y',
    'rotation 1' :180,
    'x'          :-xspan - LS24/2,
    'y'          :+Wms/6,
    'z'          :0,
    'base angle' :90,
    'base width' :w,
    'x span'     :xspan,
    'y span'     :yspan,
    'base height':thickness
})

fdtd.unselectall()
fdtd.addobject('s_bend_wg') 
fdtd.addtogroup('Input Bends') 
fdtd.set({'name' :'bend down',
    'first axis' :'z',
    'rotation 1' :180,
    'x'          :-xspan - LS24/2,
    'y'          :-Wms/6,
    'z'          :0,
    'base angle' :270,
    'base width' :w,
    'x span'     :xspan,
    'y span'     :yspan,
    'base height':thickness
})

# Creating Input extensions
fdtd.addrect() 
fdtd.addtogroup('Input Extensions') 
fdtd.set({
    'name':'rect input up',
    'x min': -xspan - LS24/2,
    'x max': - LS24/2,
    'y':Wms/6,
    'y span':w,
    'z':0,
    'z span':thickness,
})

fdtd.set('material', core)

fdtd.addrect()
fdtd.addtogroup('Input Extensions') 
fdtd.set({
    'name':'rect input down',
    'x min': -xspan - LS24/2,
    'x max': - LS24/2,
    'y':-Wms/6,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)

fdtd.addrect()
fdtd.addtogroup('Input Extensions') 
fdtd.set({
    'name':'rect bend up',
    'x min': -3*xspan - LS24/2,
    'x max': -2*xspan - LS24/2,
    'y':Wms/6 + yspan,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)

fdtd.addrect()
fdtd.addtogroup('Input Extensions') 
fdtd.set({
    'name':'rect bend down',
    'x min': -3*xspan - LS24/2,
    'x max': -2*xspan - LS24/2,
    'y':-Wms/6 - yspan,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)

# Creating Output ports

fdtd.addrect()
fdtd.addtogroup('Output Ports') 
fdtd.set({
    'name':'port1',
    'x min': LS24/2,
    'x max': LS24/2 + LM22 + xspan,
    'y': Wmf/4 + gap + w/2,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)

fdtd.addrect()
fdtd.addtogroup('Output Ports') 
fdtd.set({
    'name':'port2',
    'x min': LS24/2,
    'x max': LS24/2 + LM22 + 2*xspan,
    'y': Wmf/4 - gap - w/2,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)

fdtd.addrect()
fdtd.addtogroup('Output Ports') 
fdtd.set({
    'name':'port3',
    'x min': LS24/2 + LM22,
    'x max': LS24/2 + LM22 + 2*xspan,
    'y': -Wmf/4 + gap + w/2,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)

fdtd.addrect()
fdtd.addtogroup('Output Ports') 
fdtd.set({
    'name':'port4',
    'x min': LS24/2 + LM22,
    'x max': LS24/2 + LM22 + xspan,
    'y': -Wmf/4 - gap - w/2,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)

# Creating output bends 
fdtd.unselectall()
fdtd.addobject('s_bend_wg') 
fdtd.addtogroup('Output Bends') 
fdtd.set({'name' :'bend1',
    'first axis' :'z',
    'rotation 1' :0,
    'x span': xspan,
    'x': LS24/2 + LM22 + xspan,
    'y'          :Wmf/4 + gap + w/2,
    'z'          :0,
    'base angle' :90,
    'base width' :w,
    'y span'     :yspan,
    'base height':thickness
})

fdtd.unselectall()
fdtd.addobject('s_bend_wg') 
fdtd.addtogroup('Output Bends') 
fdtd.set({'name' :'bend2',
    'first axis' :'z',
    'rotation 1' :0,
    'x'          :LS24/2 + LM22 + 2*xspan,
    'y'          :Wmf/4 - gap - w/2,
    'z'          :0,
    'base angle' :180,
    'base width' :w,
    'x span'     :xspan,
    'y span'     :yspan,
    'base height':thickness
})

fdtd.unselectall()
fdtd.addobject('s_bend_wg') 
fdtd.addtogroup('Output Bends') 
fdtd.set({'name' :'bend3',
    'first axis' :'y',
    'rotation 1' :180,
    'x span': xspan,
    'x': LS24/2 + LM22 + 3*xspan,
    'y'          :-Wmf/4 + gap + w/2 - yspan,
    'z'          :0,
    'base angle' :90,
    'base width' :w,
    'y span'     :yspan,
    'base height':thickness
})

fdtd.unselectall()
fdtd.addobject('s_bend_wg') 
fdtd.addtogroup('Output Bends') 
fdtd.set({'name' :'bend4',
    'first axis' :'y',
    'rotation 1' :180,
    'x'          :LS24/2 + LM22 + 2*xspan,
    'y'          :-Wmf/4 - gap - w/2 - yspan,
    'z'          :0,
    'base angle' :90,
    'base width' :w,
    'x span'     :xspan,
    'y span'     :yspan,
    'base height':thickness
})

# Creating output extensions

fdtd.addrect()
fdtd.addtogroup('Output Extensions') 
fdtd.set({
    'name':'extension1',
    'x': LS24/2 + LM22 + 2.5*xspan,
    'x span' : xspan,
    'y'    :Wmf/4 + gap + w/2 + yspan,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)

fdtd.addrect()
fdtd.addtogroup('Output Extensions') 
fdtd.set({
    'name':'extension2',
    'x': LS24/2 + LM22 + 3.5*xspan,
    'x span' : xspan,
    'y'    :Wmf/4 - gap - w/2 + yspan,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)

# Creating output extensions

fdtd.addrect()
fdtd.addtogroup('Output Extensions') 
fdtd.set({
    'name':'extension3',
    'x': LS24/2 + LM22 + 3.5*xspan,
    'x span' : xspan,
    'y'    : -Wmf/4 + gap + w/2 - yspan,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)

fdtd.addrect()
fdtd.addtogroup('Output Extensions') 
fdtd.set({
    'name':'extension4',
    'x': LS24/2 + LM22 + 2.5*xspan,
    'x span' : xspan,
    'y'    :-Wmf/4 - gap - w/2 - yspan,
    'y span':w,
    'z':0,
    'z span':thickness,
})
fdtd.set('material', core)